In [1]:
pip install ultralytics

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install opencv-python mediapipe

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [4]:
import cv2
import mediapipe as mp
import time
import tkinter as tk
from tkinter import simpledialog
import threading
import tensorflow as tf
import numpy as np
from ultralytics import YOLO  # Import YOLOv8
import warnings

# Suppress TensorFlow warnings
warnings.filterwarnings("ignore", category=UserWarning, module="tensorflow")

# Initialize Mediapipe and YOLO models
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils
yolo_model = YOLO('yolov8n.pt')

# Load pre-trained CNN model
cnn_model = tf.keras.models.load_model("focused_distracted_model.h5")

# Global variables
class_duration = 0
distraction_time = 0
face_detected = False
distraction_start = None
interval = 0
distraction_threshold = 0
class_end_time = 0
pop_up_response = None
yes_count = 0


In [5]:
def get_class_duration():
    global class_duration, interval, distraction_threshold, class_end_time
    root = tk.Tk()
    root.withdraw()  
    class_duration = int(simpledialog.askinteger("Input", "Enter class duration in minutes:"))
    if class_duration:
        class_duration *= 60  
        interval = class_duration / 4
        distraction_threshold = class_duration * 0.25
        class_end_time = time.time() + class_duration
        start_monitoring()


In [6]:
def ask_question():
    global pop_up_response, yes_count, distraction_start, distraction_time

    def on_yes():
        global pop_up_response, yes_count, distraction_start, distraction_time
        pop_up_response = "Yes"
        yes_count += 1
       
        if distraction_start is not None:
            distraction_time += time.time() - distraction_start
            distraction_start = None
        root.destroy()

    def check_timeout():
        global pop_up_response, distraction_start
        if pop_up_response is None:
           
            if distraction_start is None:
                distraction_start = time.time()

    # Create a pop-up window
    root = tk.Tk()
    root.title("Student Monitoring")
    root.geometry("300x150")
    root.eval('tk::PlaceWindow . center')  

    tk.Label(root, text="Are you here?", font=("Arial", 14)).pack(pady=20)
    tk.Button(root, text="Yes", command=on_yes, width=10).pack(pady=20)

   
    root.after(15000, check_timeout)

    
    root.mainloop()

In [7]:
def multitasking_check():
    while time.time() < class_end_time:
        time.sleep(interval)  
        ask_question()  
        if pop_up_response == "No":
            print("Student did not respond in time!")


In [8]:
def start_monitoring():
    global distraction_time, face_detected, distraction_start
    cap = cv2.VideoCapture(0)
    face_mesh = mp_face_mesh.FaceMesh()

    # Start the multitasking check in a separate thread
    threading.Thread(target=multitasking_check, daemon=True).start()

    while cap.isOpened():
        current_time = time.time()
        if current_time >= class_end_time:  # Check if class duration has ended
            break

        ret, frame = cap.read()
        if not ret:
            break

        # YOLOv8: Check if a person is in the frame
        yolo_results = yolo_model(frame)
        person_detected = False
        for result in yolo_results:
            boxes = result.boxes
            for box in boxes:
                cls = int(box.cls[0])
                label = result.names[cls]
                if label == 'person':  # Check if YOLOv8 detects a person
                    person_detected = True
                    break

        if not person_detected:
            if distraction_start is None:
                distraction_start = time.time()
            distraction_time += time.time() - distraction_start
            distraction_start = time.time()
            print("No person detected - Adding to distraction time")
            continue

        # Mediapipe for face detection and CNN for classification
        process_frame_with_cnn(frame, face_mesh)

        # Display the frame
        cv2.imshow("Student Monitoring", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    show_result()


In [9]:
def process_frame_with_cnn(frame, face_mesh):
    global distraction_time, distraction_start, face_detected

    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(image)

    if results.multi_face_landmarks:
        face_detected = True
        if distraction_start:
            distraction_time += time.time() - distraction_start
            distraction_start = None

        for face_landmarks in results.multi_face_landmarks:
            mp_drawing.draw_landmarks(frame, face_landmarks, mp_face_mesh.FACEMESH_CONTOURS)

        # Preprocess the face for CNN
        face_resized = cv2.resize(frame, (224, 224))
        face_normalized = face_resized / 255.0
        face_input = np.expand_dims(face_normalized, axis=0)

        # Predict focus/distracted
        confidence = cnn_model.predict(face_input)[0][0]
        if confidence > 0.5:
            print(f"Prediction: Distracted (Confidence: {confidence:.2f})")
            if distraction_start is None:
                distraction_start = time.time()
        else:
            print(f"Prediction: Focused (Confidence: {confidence:.2f})")
            if distraction_start:
                distraction_time += time.time() - distraction_start
                distraction_start = None
    else:
        if not face_detected and distraction_start is None:
            distraction_start = time.time()
        if distraction_start:
            distraction_time += time.time() - distraction_start
            distraction_start = time.time()
        face_detected = False
        print("No face detected - Student is distracted")


In [10]:
def show_result():
    global distraction_time, distraction_threshold, yes_count
    distraction_minutes = distraction_time / 60  
    status = "Present" if distraction_time < distraction_threshold else "Absent"

    root = tk.Tk()
    root.title("Monitoring Result")
    root.geometry("400x200")
    root.eval('tk::PlaceWindow . center')

    tk.Label(root, text="Monitoring Complete", font=("Arial", 16)).pack(pady=10)
    tk.Label(root, text=f"Student Status: {status}", font=("Arial", 14)).pack(pady=5)
    tk.Label(root, text=f"Distraction Time: {distraction_minutes:.2f} minutes", font=("Arial", 14)).pack(pady=5)
    tk.Label(root, text=f"Questions Answered 'Yes': {yes_count}", font=("Arial", 14)).pack(pady=5)

    tk.Button(root, text="Close", command=root.destroy, width=10).pack(pady=20)
    root.mainloop()


In [ ]:
if __name__ == "__main__":
    get_class_duration()



0: 480x640 1 person, 181.5ms
Speed: 3.0ms preprocess, 181.5ms inference, 2.5ms postprocess per image at shape (1, 3, 480, 640)


C:\Users\fayzt\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
Prediction: Focused (Confidence: 0.00)

0: 480x640 1 person, 166.0ms
Speed: 5.0ms preprocess, 166.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Prediction: Focused (Confidence: 0.00)

0: 480x640 1 person, 161.2ms
Speed: 3.0ms preprocess, 161.2ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Prediction: Focused (Confidence: 0.00)

0: 480x640 1 person, 156.5ms
Speed: 4.0ms preprocess, 156.5ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Prediction: Focused (Confidence: 0.00)

0: 480x640 1 person, 152.5ms
Speed: 2.0ms preprocess, 152.5ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Prediction: Focused (Confidence: 0.00)

0: 480x640 1 person, 153.9ms
Speed: 3.5ms preprocess, 153.9ms inference, 1.0ms postprocess per image 

0: 480x640 1 person, 140.9ms
Speed: 2.5ms preprocess, 140.9ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Prediction: Distracted (Confidence: 1.00)

0: 480x640 1 person, 155.5ms
Speed: 3.0ms preprocess, 155.5ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Prediction: Distracted (Confidence: 1.00)

0: 480x640 1 person, 148.4ms
Speed: 3.0ms preprocess, 148.4ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Prediction: Distracted (Confidence: 1.00)

0: 480x640 1 person, 1 tie, 151.9ms
Speed: 3.0ms preprocess, 151.9ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
Prediction: Distracted (Confidence: 1.00)

0: 480x640 1 person, 150.3ms
Speed: 3.0ms preprocess, 150.3ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/s

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Prediction: Focused (Confidence: 0.00)

0: 480x640 1 person, 154.4ms
Speed: 2.0ms preprocess, 154.4ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Prediction: Focused (Confidence: 0.00)

0: 480x640 1 person, 140.8ms
Speed: 3.0ms preprocess, 140.8ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
Prediction: Focused (Confidence: 0.00)

0: 480x640 1 person, 160.9ms
Speed: 3.0ms preprocess, 160.9ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Prediction: Focused (Confidence: 0.08)

0: 480x640 1 person, 161.5ms
Speed: 2.0ms preprocess, 161.5ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Prediction: Focused (Confidence: 0.35)

0: 480x640 1 person, 151.9ms
Speed: 3.5ms preprocess, 151.9ms inference, 2.0ms postprocess per image a

...

0: 480x640 1 person, 166.5ms
Speed: 3.5ms preprocess, 166.5ms inference, 2.5ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
Prediction: Distracted (Confidence: 1.00)

0: 480x640 1 person, 158.3ms
Speed: 3.6ms preprocess, 158.3ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Prediction: Distracted (Confidence: 1.00)

0: 480x640 1 person, 142.9ms
Speed: 2.5ms preprocess, 142.9ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Prediction: Distracted (Confidence: 1.00)

0: 480x640 1 person, 164.0ms
Speed: 3.0ms preprocess, 164.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
Prediction: Distracted (Confidence: 1.00)

0: 480x640 1 person, 1 keyboard, 161.3ms
Speed: 3.6ms preprocess, 161.3ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 4

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Prediction: Focused (Confidence: 0.00)

0: 480x640 1 person, 1 tie, 154.4ms
Speed: 2.5ms preprocess, 154.4ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Prediction: Focused (Confidence: 0.00)

0: 480x640 1 person, 152.4ms
Speed: 3.6ms preprocess, 152.4ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Prediction: Focused (Confidence: 0.00)

0: 480x640 1 person, 148.9ms
Speed: 3.0ms preprocess, 148.9ms inference, 2.5ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Prediction: Focused (Confidence: 0.00)

0: 480x640 1 person, 1 tie, 145.9ms
Speed: 2.6ms preprocess, 145.9ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Prediction: Focused (Confidence: 0.00)

0: 480x640 1 person, 145.9ms
Speed: 2.0ms preprocess, 145.9ms inference, 2.0ms postproce

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
Prediction: Focused (Confidence: 0.10)

0: 480x640 1 person, 143.9ms
Speed: 2.5ms preprocess, 143.9ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Prediction: Focused (Confidence: 0.13)

0: 480x640 1 person, 147.0ms
Speed: 2.0ms preprocess, 147.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Prediction: Focused (Confidence: 0.11)

0: 480x640 1 person, 142.4ms
Speed: 3.0ms preprocess, 142.4ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Prediction: Focused (Confidence: 0.10)

0: 480x640 1 person, 141.9ms
Speed: 3.5ms preprocess, 141.9ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Prediction: Focused (Confidence: 0.10)

0: 480x640 1 person, 139.9ms
Speed: 3.0ms preprocess, 139.9ms inference, 2.0ms postprocess per image a

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Prediction: Focused (Confidence: 0.12)

0: 480x640 1 person, 157.6ms
Speed: 3.0ms preprocess, 157.6ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Prediction: Focused (Confidence: 0.10)

0: 480x640 1 person, 151.9ms
Speed: 3.0ms preprocess, 151.9ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
Prediction: Focused (Confidence: 0.13)

0: 480x640 1 person, 163.5ms
Speed: 4.6ms preprocess, 163.5ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
Prediction: Focused (Confidence: 0.03)

0: 480x640 1 person, 159.2ms
Speed: 3.0ms preprocess, 159.2ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Prediction: Focused (Confidence: 0.03)

0: 480x640 1 person, 146.9ms
Speed: 3.1ms preprocess, 146.9ms inference, 1.5ms postprocess per image a

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Prediction: Distracted (Confidence: 0.98)

0: 480x640 1 person, 151.4ms
Speed: 3.0ms preprocess, 151.4ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Prediction: Distracted (Confidence: 1.00)

0: 480x640 1 person, 152.9ms
Speed: 3.0ms preprocess, 152.9ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Prediction: Distracted (Confidence: 1.00)

0: 480x640 1 person, 152.7ms
Speed: 4.5ms preprocess, 152.7ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Prediction: Distracted (Confidence: 1.00)

0: 480x640 1 person, 1 laptop, 148.8ms
Speed: 3.0ms preprocess, 148.8ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Prediction: Distracted (Confidence: 1.00)

0: 480x640 1 person, 149.9ms
Speed: 3.0ms preprocess, 149.9ms inference, 2.0m